In [ ]:
import requests
import tweepy
import pandas as pd
import math
import json
from datetime import datetime as dt
from datetime import timezone as tz

In [ ]:
CREDENTIALS_PATH = "./credentials.json"
credentials = ""
with open(CREDENTIALS_PATH, "r") as cred:
    credentials = json.load(cred)
    
client = tweepy.Client(**credentials)

In [ ]:
df = pd.read_csv("../data/source/congress_member_data.csv")
congress_twitter_handles = [x for x in list(df["twitter"]) if type(x) == str]

In [ ]:
# Static parameters for tweet search
fields = ['created_at', 'geo', 'lang']
start_utc = dt.strptime("01/01/2020 00:00:00", "%m/%d/%Y %H:%M:%S")
end_utc = dt.strptime("12/31/2020 11:59:59", "%m/%d/%Y %H:%M:%S")
start_utc_iso8601 = start_utc.astimezone(tz.utc).isoformat('T', timespec='milliseconds').replace("+00:00", "Z")
end_utc_iso8601 = end_utc.astimezone(tz.utc).isoformat('T', timespec='milliseconds').replace("+00:00", "Z")

# Formatted dictionary to 
users_tweets_kwargs = {
    "max_results": 100,
    "start_time": start_utc_iso8601,
    "end_time": end_utc_iso8601,
    "tweet_fields": fields
}

In [ ]:
def write_to_csv(filename, dictionary):
    with open(filename, "w") as f:
        for key, value in dictionary.items():
            if len(value) != 0:
                for val in value:
                    text = val.text.replace('"', '\\"').replace("'", "\\'").replace("\n", "").replace(",", "").strip()
                    text = "\"" + text + "\""
                    items = [key, text, str(val.id), str(val.geo), str(val.lang), str(val.created_at)]
                    line = ",".join(items) + "\n"
                    f.write(line)
            else:
                items = [key, "", "", "", ""]
                line = ",".join(items) + "\n"
                f.write(line)

In [ ]:
def get_tweets(client, handles, tweet_details, checked_handles, dictionary={}):
    checked = open(cache, "r+")
#     checked_handles = [line for line in checked.read().split("\n") if line != ""]
    
    print("There are", 522 - len(checked_handles), "to check")
    # Looping through all twitter handles to get tweets
    for handle in handles:
        if handle not in checked_handles:
            congress_person = client.get_user(username=handle)
            if congress_person.data != None:
                tweet_details["id"] = congress_person.data.id
                paginator = tweepy.Paginator(client.get_users_tweets, **tweet_details)
                dictionary[handle] = []

                # Iterate through pages of 100 tweets and append each tweet
                for response in paginator:
                    if (response.data) and (len(response.data) > 0):
                        for tweet in response.data:
                            dictionary[handle].append(tweet)
            else:
                dictionary[handle] = []

            # Writing to csv, one handle at a time
            filename = "../data/" + "congress_tweets_" + handle + ".csv"
            write_to_csv(filename, dictionary)
            
            # TODO: Fix checked_handles not being written to txt file
            with open(cache, "w+") as c:
                c.write(handle + "\n")
            
            checked_handles.append(handle)
            dictionary = {}
            print("Finished", handle)
        else:
            print("Already Checked", handle)

    return checked_handles

In [ ]:
cache = "../data/checked_handles_new.txt"
checked = open(cache, "r+")
checked_handles = [line for line in checked.read().split("\n") if line != ""]
checked.close()

In [ ]:
print(len(checked_handles))

In [ ]:
get_tweets(client, congress_twitter_handles, users_tweets_kwargs, checked_handles)